In [38]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\adria\AppData\Local\Temp\ipykernel_30992\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab. Natural Lenguage Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [40]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speead up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [41]:
train_data = pd.read_csv("data/kg_train.csv", encoding='latin-1')
test_data = pd.read_csv("data/kg_test.csv", encoding='latin-1')

# Reduce the training set 
train_data = train_data.head(1000)


train_data.fillna("", inplace=True)
test_data.fillna("", inplace=True)


print("Training set shape:", train_data.shape)
print("Test set shape:", test_data.shape)


Training set shape: (1000, 2)
Test set shape: (5964, 1)


## Data Preprocessing

In [42]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [43]:
import re
from bs4 import BeautifulSoup


def clean_html(raw_html):
    # Remove inline JavaScript/CSS
    cleaned_html = re.sub(r'<(script|style).*?>.*?</\1>', '', raw_html, flags=re.DOTALL)
    
    # Remove HTML comments
    cleaned_html = re.sub(r'<!--.*?-->', '', cleaned_html, flags=re.DOTALL)
    
    # Remove remaining HTML tags
    cleaned_html = BeautifulSoup(cleaned_html, "html.parser").get_text()
    
    return cleaned_html

# Preprocessing
def preprocess_text(text):
    # Clean HTML content
    text = clean_html(text)
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    #lowercase
    text = text.lower()
    
    return text

# Apply preprocessing to textcolumn
train_data['text'] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)

print(train_data.head())
print(test_data.head())


C:\Users\adria\AppData\Local\Temp\ipykernel_30992\4106381147.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned_html = BeautifulSoup(cleaned_html, "html.parser").get_text()
C:\Users\adria\AppData\Local\Temp\ipykernel_30992\4106381147.py:13: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  cleaned_html = BeautifulSoup(cleaned_html, "html.parser").get_text()


                                                text  label
0  dear sir strictly a private business proposal ...      1
1                                            will do      0
2  noracheryl has emailed dozens of memos about h...      0
3  dear sir2fmadam2c i know that this proposal mi...      1
4                                                fyi      0
                                                text
0  usiness is for the fact that the deceased man ...
1  they are happy to adjust to the afternoon i am...
2  lael brainard was confirmed 7819 this afternoo...
3  h friday march 26 2010 545 amsbwhoeopãâ¢ãâã...
4  n dear good friendi am happy to inform you abo...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [44]:
# Preprocessing functions 2
def preprocess_text_1(text):
    # Clean HTML content
    text = clean_html(text)
    
    # Remove all special characters
    text = re.sub(r'\W', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d', '', text)
    
    # Remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Remove prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Apply preprocessing to text column
train_data['text'] = train_data['text'].apply(preprocess_text_1)
test_data['text'] = test_data['text'].apply(preprocess_text_1)

print(train_data.head())
print(test_data.head())


                                                text  label
0  dear sir strictly private business proposal am...      1
1                                            will do      0
2  noracheryl has emailed dozens of memos about h...      0
3  dear sirfmadamc know that this proposal might ...      1
4                                                fyi      0
                                                text
0  usiness is for the fact that the deceased man ...
1  they are happy to adjust to the afternoon am g...
2  lael brainard was confirmed this afternoonmigu...
3  h friday march amsbwhoeopã â ã â ã â rei have ...
4  n dear good friendi am happy to inform you abo...


## Now let's work on removing stopwords
Remove the stopwords.

In [45]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

train_data['text'] = train_data['text'].apply(remove_stopwords)
test_data['text'] = test_data['text'].apply(remove_stopwords)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [46]:
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return lemmatized_text

train_data['text'] = train_data['text'].apply(lemmatize_text)
test_data['text'] = test_data['text'].apply(lemmatize_text)


In [47]:
print(train_data.head())

                                                text  label
0  dear sir strictly private business proposal mi...      1
1                                                         0
2  noracheryl emailed dozen memo haiti weekend pl...      0
3  dear sirfmadamc know proposal might surprise e...      1
4                                                fyi      0


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [48]:
from collections import Counter

# Tokenize the text data
ham_messages = train_data[train_data['label'] == 0]['text'].apply(lambda x: x.split())
spam_messages = train_data[train_data['label'] == 1]['text'].apply(lambda x: x.split())

# Count the occurrences of each word
ham_word_counts = Counter(word for message in ham_messages for word in message)
spam_word_counts = Counter(word for message in spam_messages for word in message)

#top 10 words
top_10_ham_words = ham_word_counts.most_common(10)
top_10_spam_words = spam_word_counts.most_common(10)

print("Top 10 words in ham messages:")
for word, count in top_10_ham_words:
    print(f"{word}: {count}")

print("\nTop 10 words in spam messages:")
for word, count in top_10_spam_words:
    print(f"{word}: {count}")

Top 10 words in ham messages:
â: 244
u: 119
pm: 115
would: 106
state: 103
president: 94
call: 91
time: 84
percent: 77
secretary: 76

Top 10 words in spam messages:
money: 920
account: 795
bank: 745
fund: 705
u: 550
business: 474
transaction: 416
country: 407
transfer: 393
million: 386


## Extra features

In [49]:
money_symbol_list = ["euro", "dollar", "pound", "€", "$"]
suspicious_words = ["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"]

# Convert the lists into regular expression patterns
money_symbol_pattern = "|".join(money_symbol_list)
suspicious_words_pattern = "|".join(suspicious_words)

# Define functions to check for the moneu symbols and sus words
def has_money_symbol(text):
    return int(bool(re.search(money_symbol_pattern, text)))

def has_suspicious_words(text):
    return int(bool(re.search(suspicious_words_pattern, text)))

# Add indicators to the original dataframe
train_data['money_mark'] = train_data['text'].apply(has_money_symbol)
train_data['suspicious_words'] = train_data['text'].apply(has_suspicious_words)
train_data['text_len'] = train_data['text'].apply(len)

print(train_data.head())

                                                text  label  money_mark  \
0  dear sir strictly private business proposal mi...      1           1   
1                                                         0           1   
2  noracheryl emailed dozen memo haiti weekend pl...      0           1   
3  dear sirfmadamc know proposal might surprise e...      1           1   
4                                                fyi      0           1   

   suspicious_words  text_len  
0                 1      1504  
1                 0         0  
2                 0       110  
3                 1      1380  
4                 0         3  


## How would work the Bag of Words with Count Vectorizer concept?

In [50]:
from sklearn.feature_extraction.text import CountVectorizer


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [51]:
vectorizer = CountVectorizer()

#fit the vectoriser to the trian_data and transform
X = vectorizer.fit_transform(train_data)

#feature names
feature_names = vectorizer.get_feature_names_out()


In [52]:
print("Bag-of-Words Representation:")
print(X.toarray())

print("\nFeature Names:")
print(feature_names)

Bag-of-Words Representation:
[[0 0 0 1 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]]

Feature Names:
['label' 'money_mark' 'suspicious_words' 'text' 'text_len']


In [53]:
# shape of the vetorized dataset
print("Shape of the vectorized dataset:", X.shape)

Shape of the vectorized dataset: (5, 5)


## And the Train a Classifier?

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['text'])
X_test = vectorizer.transform(test_data['text'])

# Define labels
y_train = train_data['label']

# Create a Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = classifier.predict(X_test)


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [55]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

data = pd.read_csv("data/kg_train.csv", encoding='latin-1')


#data preprocess here
data.fillna("", inplace=True)
data['text'] = data['text'].apply(preprocess_text).apply(preprocess_text_1).apply(remove_stopwords).apply(lemmatize_text)


X = data['text'] 
y = data['label'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()  # You can experiment with different vectorizers
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Create and train the MultinomialNB classifier
classifier = MultinomialNB()
classifier.fit(X_train_vectorized, y_train)

y_pred = classifier.predict(X_test_vectorized)

#eval
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))


C:\Users\adria\AppData\Local\Temp\ipykernel_30992\4106381147.py:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned_html = BeautifulSoup(cleaned_html, "html.parser").get_text()
C:\Users\adria\AppData\Local\Temp\ipykernel_30992\4106381147.py:13: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  cleaned_html = BeautifulSoup(cleaned_html, "html.parser").get_text()


Accuracy: 0.9471919530595139
              precision    recall  f1-score   support

           0       0.97      0.94      0.95       674
           1       0.92      0.96      0.94       519

    accuracy                           0.95      1193
   macro avg       0.95      0.95      0.95      1193
weighted avg       0.95      0.95      0.95      1193

